Mahesh Rao


Jan 22, 2020

Apologies, this notebook is very messy. I'll try to clean it up over time. Basically this contains code I used to generate the database file to make an ERD for data question 2 for the analytics jumpstart

In [1]:
import pandas as pd
import string
import sqlite3 as sql
from sqlalchemy import MetaData, Table, Column, Integer, String, Float, DateTime, ForeignKey, create_engine, insert
from eralchemy import render_er
import numpy as np
from scipy.stats import mode

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
permits = pd.read_csv('data/Building_Permits_Issued.csv', low_memory=False)
short_term_permits = pd.read_csv('data/Residential_Short_Term_Rental_Permits.csv')

FileNotFoundError: [Errno 2] File b'data/Building_Permits_Issued.csv' does not exist: b'data/Building_Permits_Issued.csv'

In [4]:
short_term_permits.columns

NameError: name 'short_term_permits' is not defined

In [5]:
def make_table_input(df, table_name, primary_key = None, foreign_key = None):
    '''
    Function to print out the function input to make a sqlalchemy table
    
    foreign_key arg requires a list of tuples or lists with 2 elements: (col_name, reference_df.reference_col_name)
    '''
    column_list = []
    for c in df.columns:
        arg = "Column('{}', {}".format(c, df[c].dtype)
        if foreign_key is not None:
            for f_key in foreign_key:
                if f_key[0]==c:
                    arg += ", ForeignKey('{}')".format(f_key[1])
        if primary_key == c:
            arg += ", primary_key=True"
        arg += ")"
        column_list.append(arg)
    concat_args = ", \n".join(column_list).replace('object', 'String').replace('float64', 'Float').replace('int64', 'Integer').replace('datetime64[ns]', 'DateTime')
    return "Table('{}', metadata, \n".format(table_name)+concat_args+')'

In [6]:
print(make_table_input(permits, 'permits', primary_key = 'Permit #'))

NameError: name 'permits' is not defined

In [7]:
print(make_table_input(short_term_permits, 'short_term', primary_key = 'Permit #', foreign_key = ('Permit #', 'permits.Permit #', )))

NameError: name 'short_term_permits' is not defined

In [8]:
from sqlalchemy import (MetaData, Table, Column, Integer, String, Float, ForeignKey) 

metadata = MetaData()

permits_table = Table('permits', metadata, 
                      Column('Permit #', String, primary_key=True), 
                      Column('Permit Type Description', String), 
                      Column('Permit Subtype Description', String), 
                      Column('Parcel', String), 
                      Column('Date Entered', String), 
                      Column('Date Issued', String), 
                      Column('Construction Cost', Float), 
                      Column('Address', String), 
                      Column('City', String), 
                      Column('State', String), 
                      Column('ZIP', Integer), 
                      Column('Subdivision / Lot', String), 
                      Column('Contact', String), 
                      Column('Permit Type', String), 
                      Column('Permit Subtype', String), 
                      Column('IVR Tracking #', Integer), 
                      Column('Purpose', String), 
                      Column('Council District', Float), 
                      Column('Census Tract', Float), 
                      Column('Mapped Location', String))

In [9]:
short_term_table = Table('short_term', metadata, 
                         Column('Permit #', String, ForeignKey('permits.Permit #'), primary_key=True), 
                         Column('Applicant', String), 
                         Column('Applicant Home Phone', String), 
                         Column('Applicant Mobile Phone', String), 
                         Column('Applicant Work Phone', String), 
                         Column('Applicant Work Phone Ext.', String), 
                         Column('Contact', String), 
                         Column('Contact Home Phone', String), 
                         Column('Contact Mobile Phone', String), 
                         Column('Contact Work Phone', String), 
                         Column('Contact Work Phone Ext.', String), 
                         Column('Permit Subtype Description', String), 
                         Column('Permit Status', String), 
                         Column('Parcel', String), 
                         Column('Date Entered', String), 
                         Column('Date Issued', String), 
                         Column('Expiration Date', String), 
                         Column('Address', String), 
                         Column('City', String), 
                         Column('State', String), 
                         Column('ZIP', Integer), 
                         Column('Subdivision / Lot', String), 
                         Column('Purpose', String), 
                         Column('Permit Owner Name', String), 
                         Column('Permit Owner Addr. 1', String), 
                         Column('Permit Owner Addr. 2', String), 
                         Column('Permit Owner Addr. 3', String), 
                         Column('Permit Owner City', String), 
                         Column('Permit Owner State', String), 
                         Column('Permit Owner ZIP', String), 
                         Column('Permit Type', String), 
                         Column('Permit Subtype', String), 
                         Column('Council District', Float), 
                         Column('Census Tract', Integer), 
                         Column('Mapped Location', String))

---

In [2]:
# download from https://data.nashville.gov/General-Government/Metro-Resident-Survey-Responses/nux7-cwab
survey = pd.read_csv('../../../Jumpstart_new_dataquestion_1-13-20/extra_data/Metro_Resident_Survey_Responses.csv', low_memory=False)
# download from https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7
permits = pd.read_csv('../../../Jumpstart_new_dataquestion_1-13-20/permits_erd_test.csv', low_memory=False, index_col = 0)
# download from https://data.nashville.gov/Fire/Fire-Stations/frq9-a5iv
stations_district = pd.read_csv('../../../Jumpstart_new_dataquestion_1-13-20/stations_erd_test.csv', low_memory=False, index_col = 0)
# download from https://data.nashville.gov/Licenses-Permits/Residential-Short-Term-Rental-Permits/2z82-v8pm
short = pd.read_csv('../../../Jumpstart_new_dataquestion_1-13-20/short_erd_test.csv', low_memory=False, index_col = 0)

In [3]:
survey = survey.reset_index().rename(columns = {'index':'id'})
new_col = []
for i in survey.columns:
    new_i = i.lower().strip().replace('-', ' ')
    new_i = new_i.translate(str.maketrans('', '', string.punctuation)).replace('  ', ' ').replace(' ', '_')
    new_col.append(new_i)
survey.columns = new_col
survey_fire = survey[['id', 'zip_code', 'period', 'start_date', 'end_date', 'short_term_rental_enforcement', 'city_appearance', 'neighborhood_appearance', 'nashville_as_residence', 'nashville_as_place_to_raise_children','nashville_as_workplace','nashville_as_place_to_startgrow_business','nashville_as_retirement_place']+[c for c in survey.columns if 'fire' in c]]

In [4]:
survey_fire_2018 = survey_fire[survey_fire['period'].str.contains('2018Q3')]

In [5]:
mode([1, 2, 3, 2, 3, 2, 3, 3])[0][0]

3

In [6]:
pd.DataFrame(dict(zip(['test_1', 'test_2'], [0, 2])), [37216])

,test_1,test_2
37216,0,2


In [7]:
len(survey_fire_2018['zip_code'].unique())

33

In [8]:
survey_fire_2018_single_list = []
for g, df in list(survey_fire_2018.groupby('zip_code')):
    new_val_list = []
    for c in survey_fire_2018.columns:
        new_val_list.append(mode(df[c])[0][0])
    survey_fire_2018_single_list.append(pd.DataFrame(dict(zip(survey_fire_2018.columns, new_val_list)), [0]))
survey_fire_2018_single = pd.concat(survey_fire_2018_single_list)

In [9]:
# Final version
survey_fire_2018_single = survey_fire_2018_single.reset_index(drop=True)

In [10]:
survey_fire_2018_single

,id,zip_code,period,start_date,end_date,short_term_rental_enforcement,city_appearance,neighborhood_appearance,nashville_as_residence,nashville_as_place_to_raise_children,nashville_as_workplace,nashville_as_place_to_startgrow_business,nashville_as_retirement_place,fire_and_ems__overall,fire_services,fire_response_time,fire_professionalism
0,813,37013,2018Q3,09/06/2018,10/17/2018,Don't Know,Satisfied,Satisfied,Good,Good,Good,Good,Neutral,Satisfied,Satisfied,Satisfied,Satisfied
1,885,37015,2018Q3,09/06/2018,10/17/2018,Don't Know,Neutral,Very Satisfied,Good,Good,Good,Neutral,Good,Don't Know,Don't Know,Don't Know,Don't Know
2,815,37023,2018Q3,09/06/2018,10/17/2018,Neutral,Neutral,Dissatisfied,Below Average,Below Average,Poor,Poor,Poor,Satisfied,Satisfied,Satisfied,Satisfied
3,876,37027,2018Q3,09/06/2018,10/17/2018,Don't Know,Neutral,Satisfied,Good,Don't Know,Excellent,Below Average,Good,Satisfied,Satisfied,Satisfied,Very Satisfied
4,879,37072,2018Q3,09/06/2018,10/17/2018,Neutral,Satisfied,Satisfied,Good,Neutral,Excellent,Neutral,Good,Very Satisfied,Satisfied,Satisfied,Very Satisfied
5,823,37076,2018Q3,09/06/2018,10/17/2018,Don't Know,Satisfied,Satisfied,Good,Neutral,Good,Good,Good,Satisfied,Satisfied,Satisfied,Satisfied
6,942,37115,2018Q3,09/06/2018,10/17/2018,Neutral,Satisfied,Satisfied,Good,Good,Don't Know,Neutral,Good,Satisfied,Satisfied,Neutral,Don't Know
7,1167,37116,2018Q3,09/06/2018,10/17/2018,Very Dissatisfied,Dissatisfied,Dissatisfied,Neutral,Below Average,Below Average,Neutral,Poor,Satisfied,Satisfied,Satisfied,Satisfied
8,1216,37135,2018Q3,09/06/2018,10/17/2018,Dissatisfied,Neutral,Very Satisfied,Good,Good,Excellent,Don't Know,Neutral,Don't Know,Dissatisfied,Don't Know,Don't Know
9,807,37138,2018Q3,09/06/2018,10/17/2018,Dissatisfied,Neutral,Neutral,Good,Good,Good,Good,Good,Satisfied,Satisfied,Satisfied,Neutral


In [11]:
# Need to figure out what to do about this
permits[permits['number']=='2017011225']

,number,type_desc,subtype_desc,parcel,enter_date,issue_date,cost,address,city,state,...,contact,type_code,subtype_code,tracking_number,purpose,council_district,census_tract,mapped_location,issue_year,lapse
10971,2017011225,Building Demolition Permit,Demolition Permit - Residential,083031F90000CO,2017-02-16,2017-02-16,5000.0,1301F PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301F PORTER RD\nNASHVILLE, TN 37206\n(36.1896...",2017,0 days 00:00:00.000000000
11067,2017011225,Building Demolition Permit,Demolition Permit - Residential,083031G00100CO,2017-02-16,2017-02-16,5000.0,1301A PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301A PORTER RD\nNASHVILLE, TN 37206\n(36.1896...",2017,0 days 00:00:00.000000000
11941,2017011225,Building Demolition Permit,Demolition Permit - Residential,083031G90000CO,2017-02-16,2017-02-16,5000.0,1301E PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301E PORTER RD\nNASHVILLE, TN 37206\n(36.1896...",2017,0 days 00:00:00.000000000
14442,2017011225,Building Demolition Permit,Demolition Permit - Residential,083031F00100CO,2017-02-16,2017-02-16,5000.0,1301C PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301C PORTER RD\nNASHVILLE, TN 37206\n(36.1896...",2017,0 days 00:00:00.000000000
27293,2017011225,Building Demolition Permit,Demolition Permit - Residential,08303014300,2017-02-16,2017-02-16,5000.0,1301 PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301 PORTER RD\nNASHVILLE, TN 37206\n(36.18966...",2017,0 days 00:00:00.000000000
31099,2017011225,Building Demolition Permit,Demolition Permit - Residential,083031G00200CO,2017-02-16,2017-02-16,5000.0,1301B PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301B PORTER RD\nNASHVILLE, TN 37206\n(36.1896...",2017,0 days 00:00:00.000000000
32721,2017011225,Building Demolition Permit,Demolition Permit - Residential,083031F00200CO,2017-02-16,2017-02-16,5000.0,1301D PORTER RD,NASHVILLE,TN,...,L & S CONSTRUCTION SERVICES,CADM,CAZ01A001,3321232,To demo existing single family residence and a...,7.0,37011400.0,"1301D PORTER RD\nNASHVILLE, TN 37206\n(36.1896...",2017,0 days 00:00:00.000000000


In [12]:
permits['num_parcel_add'] = permits['number']+permits['parcel']+permits['address'].str.split(' ').str[0]

In [13]:
permits = permits.drop_duplicates([c for c in permits.columns if c not in ['contact', 'address', 'mapped_location']])

In [14]:
permits_2018 = permits[permits['issue_year']==2018]

In [15]:
permits_2018['issue_year'].value_counts()

2018    11080
Name: issue_year, dtype: int64

In [16]:
short['id_parcel'] = short['id'].astype(str)+short['parcel'].astype(str)

In [17]:
short['id_parcel'].value_counts()

201802303911603006300       2
201700630109313010400       2
2019057706092100F01600CO    1
2019052875072063H02600CO    1
2018075592092040G00100CO    1
                           ..
2017052403093024A10600CO    1
20152491110511028600        1
2018006055104020R00800CO    1
2019032756072050F00400CO    1
2019005065105074K00200CO    1
Name: id_parcel, Length: 3360, dtype: int64

In [18]:
short[short['id_parcel']=='201802303911603006300']

,id,applicant,contact,subtype_desc,status,parcel,entered,issued,expiration,address,city,state,zip,purpose,permit_owner_name,type,council_district,id_parcel
3275,2018023039,"ANDREWS, ERNEST ALBERT III & NITA BAUGH",Nita Andrews,Short Term Rental - Owner Occupied,ISSUED,11603006300,04/24/2018 12:00:00 AM,2018-05-04,05/04/2020 12:00:00 AM,130A KENNER AVE,NASHVILLE,TN,37205,By making this application for a Residential S...,"ANDREWS, ERNEST ALBERT III & NITA BAUGH",CASR,24.0,201802303911603006300
6024,2018023039,"ANDREWS, ERNEST ALBERT III & NITA BAUGH","ANDREWS, ERNEST ALBERT III & NITA BAUGH",Short Term Rental - Owner Occupied,ISSUED,11603006300,04/24/2018 12:00:00 AM,2018-05-04,05/04/2020 12:00:00 AM,130A KENNER AVE,NASHVILLE,TN,37205,By making this application for a Residential S...,"ANDREWS, ERNEST ALBERT III & NITA BAUGH",CASR,24.0,201802303911603006300


In [19]:
short = short.drop_duplicates([c for c in short.columns if c != 'contact'])

In [20]:
short['issued'] = pd.to_datetime(short['issued'])

In [21]:
short_2018 = short[short['issued'].dt.year==2018]#.sort_values('issued')

In [22]:
stations_district

,district,station_number
0,1,['Station 30']
1,2,['Station 24' 'Station 25']
2,3,['Station 39']
3,5,['Station 3']
4,6,['Station 14']
5,7,['Station 29' 'Station 18']
6,8,['Station 1' 'Station 31']
7,9,['Station 38']
8,11,['Station 36']
9,13,['Station 15' 'Station 4']


In [23]:
for d in list(range(1, 34)):
    if d not in stations_district['district'].tolist():
#         print(d)
#         print(pd.DataFrame(data = {'district':d, 'station_number':[]}))
        stations_district = stations_district.append(pd.DataFrame(data = {'district':[d], 'station_number':['[]']}))

In [24]:
stations_district = stations_district.sort_values('district').reset_index(drop=True)

In [25]:
stations_district

,district,station_number
0,1,['Station 30']
1,2,['Station 24' 'Station 25']
2,3,['Station 39']
3,4,[]
4,5,['Station 3']
5,6,['Station 14']
6,7,['Station 29' 'Station 18']
7,8,['Station 1' 'Station 31']
8,9,['Station 38']
9,10,[]


In [26]:
print(make_table_input(permits_2018, 'permits_2018', primary_key = 'num_parcel_add', foreign_key = [('parcel', 'strp_2018.parcel'), ('council_district', 'station_dist.district'), ('zip', 'resident_survey_2018.zip_code')]))
print('\n')
print(make_table_input(short_2018, 'strp_2018', primary_key = 'id', foreign_key = [('parcel', 'permits_2018.parcel'), ('council_district', 'station_dist.district'), ('zip', 'resident_survey_2018.zip_code')]))
print('\n')
print(make_table_input(stations_district, 'station_dist', primary_key = 'district'))
print('\n')
print(make_table_input(survey_fire_2018_single, 'resident_survey_2018', primary_key = 'id'))


NameError: name 'make_table_input' is not defined

In [27]:
db_uri = 'sqlite:///multi_table_2.db'
engine = create_engine(db_uri)
connection = engine.connect()
metadata = MetaData(engine)

permits_table = Table('permits_2018', metadata, 
Column('number', String), 
Column('type_desc', String), 
Column('subtype_desc', String), 
Column('parcel', String, ForeignKey('strp_2018.parcel')), 
Column('enter_date', String), 
Column('issue_date', String), 
Column('cost', Float), 
Column('address', String), 
Column('city', String), 
Column('state', String), 
Column('zip', Integer, ForeignKey('resident_survey_2018.zip_code')), 
Column('subdivision', String), 
Column('contact', String), 
Column('type_code', String), 
Column('subtype_code', String), 
Column('tracking_number', Integer), 
Column('purpose', String), 
Column('council_district', Integer, ForeignKey('station_dist.district')), 
Column('census_tract', Float), 
Column('mapped_location', String), 
Column('issue_year', Integer), 
Column('lapse', String), 
Column('num_parcel_add', String, primary_key=True))

short_table = Table('strp_2018', metadata, 
Column('id', String, primary_key=True), 
Column('applicant', String), 
Column('contact', String), 
Column('subtype_desc', String), 
Column('status', String), 
Column('parcel', String, ForeignKey('permits_2018.parcel')), 
Column('entered', String), 
Column('issued', DateTime), 
Column('expiration', String), 
Column('address', String), 
Column('city', String), 
Column('state', String), 
Column('zip', Integer, ForeignKey('resident_survey_2018.zip_code')), 
Column('purpose', String), 
Column('permit_owner_name', String), 
Column('type', String), 
Column('council_district', Integer, ForeignKey('station_dist.district')), 
Column('id_parcel', String))

station_dist_table = Table('station_dist', metadata, 
Column('district', Integer, primary_key=True), 
Column('station_number', String))

survey_table = Table('resident_survey_2018', metadata, 
Column('id', Integer, primary_key=True), 
Column('zip_code', Integer), 
Column('period', String), 
Column('start_date', String), 
Column('end_date', String), 
Column('short_term_rental_enforcement', String), 
Column('city_appearance', String), 
Column('neighborhood_appearance', String), 
Column('nashville_as_residence', String), 
Column('nashville_as_place_to_raise_children', String), 
Column('nashville_as_workplace', String), 
Column('nashville_as_place_to_startgrow_business', String), 
Column('nashville_as_retirement_place', String), 
Column('fire_and_ems__overall', String), 
Column('fire_services', String), 
Column('fire_response_time', String), 
Column('fire_professionalism', String))

# permits_table.create()
# short_table.create()
# station_dist_table.create()
# survey_table.create()

metadata.create_all(engine)

In [28]:
# if get IntegrityError, its because multi_table_2.db already exists. Shortcut is to just delete the file and then rerun this

for table, df in zip([station_dist_table, permits_table, short_table, survey_table], [stations_district, permits_2018, short_2018, survey_fire_2018_single]):
    print(table)
    query = insert(table)
#     print(query)
    values_list = list(df.to_dict(orient='index').values())
#     print(values_list)
    ResultProxy = connection.execute(query,values_list)

station_dist
permits_2018
strp_2018
resident_survey_2018


In [29]:
filename = 'mymodel.png'
render_er(metadata, filename)

# ERD will output as mymodel.png at the same directory level as this notebook

In [30]:
db = 'multi_table_2.db'
con = sql.connect(db)
mycursor = con.cursor() 
mycursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables=(mycursor.fetchall())
print(tables)

[('permits_2018',), ('resident_survey_2018',), ('station_dist',), ('strp_2018',)]


In [31]:
survey_fire_2018_single.merge(short_2018, how = 'inner', left_on = 'zip_code', right_on = 'zip').merge(stations_district, how = 'right', left_on = 'council_district', right_on = 'district')[['district', 'station_number', 'fire_and_ems__overall', 'fire_services', 'fire_response_time', 'fire_professionalism']].drop_duplicates('district')

,district,station_number,fire_and_ems__overall,fire_services,fire_response_time,fire_professionalism
0,33,['Station 35'],Satisfied,Satisfied,Satisfied,Satisfied
3,29,['Station 33'],Satisfied,Satisfied,Satisfied,Satisfied
8,31,[],Satisfied,Satisfied,Satisfied,Very Satisfied
9,10,[],Very Satisfied,Satisfied,Satisfied,Very Satisfied
11,12,[],Satisfied,Satisfied,Satisfied,Satisfied
16,14,['Station 32'],Satisfied,Satisfied,Satisfied,Satisfied
21,11,['Station 36'],Satisfied,Satisfied,Satisfied,Satisfied
24,9,['Station 38'],Satisfied,Satisfied,Neutral,Don't Know
30,7,['Station 29' 'Station 18'],Satisfied,Satisfied,Neutral,Don't Know
63,8,['Station 1' 'Station 31'],Satisfied,Satisfied,Neutral,Don't Know


[('permits',), ('station_dist',), ('strp',), ('survey',)]
[('building_permits',), ('fire_stations',), ('resident_survey',), ('short_term_permits',)]

SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate
FROM Orders
INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID

In [32]:
db = 'multi_table_2.db'
con = sql.connect(db)
mycursor = con.cursor()
mycursor.execute("SELECT * FROM strp_2018 LEFT JOIN station_dist ON strp_2018.council_district=station_dist.district")
two_tables=(mycursor.fetchall())
#print(two_tables)

In [33]:
pd.DataFrame(two_tables)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2018024546,Jamia Richmond & Zachary Bentley,Jamia Richmond & Zachary Bentley,Short Term Rental - Owner Occupied,ISSUED,071070D00200CO,05/01/2018 12:00:00 AM,2018-05-10 00:00:00.000000,05/10/2020 12:00:00 AM,304 QUEEN AVE,NASHVILLE,TN,37207,By making this application for a Residential S...,None,CASR,5,2018024546071070D00200CO,5.0,['Station 3']
1,2018006025,Adam Klenk,"Nestive, LLC",Short Term Rental � Multifamily,ISSUED,104020R00400CO,02/01/2018 12:00:00 AM,2018-03-02 00:00:00.000000,03/02/2020 12:00:00 AM,3181 PARTHENON AVE 4,NASHVILLE,TN,37203,By making this application for a Type 3 Reside...,Adam Klenk,CASR,21,2018006025104020R00400CO,21.0,['Station 11' 'Station 19']
2,2018010131,Sarah Upton,Sarah Upton,Short Term Rental - Owner Occupied,ISSUED,092090V00200CO,02/22/2018 12:00:00 AM,2018-03-02 00:00:00.000000,03/02/2020 12:00:00 AM,420B 37TH AVE N,NASHVILLE,TN,37209,By making this application for a Type 1 Reside...,"UPTON, SARAH J. & BRANDON M.",CASR,24,2018010131092090V00200CO,24.0,['Station 13' 'Station 17']
3,2018067475,Micah Sannan,Micah Sannan,Short Term Rental - Owner Occupied,ISSUED,06115035900,10/24/2018 12:00:00 AM,2018-11-07 00:00:00.000000,11/07/2020 12:00:00 AM,3814 KATHERINE ST,NASHVILLE,TN,37216,By making this application for a Residential S...,"RINCK, AMBER & SANNAN, MICAH",CASR,7,201806747506115035900,7.0,['Station 29' 'Station 18']
4,2018070389,The Lease Killers,The Lease Killers,Short Term Rental � Multifamily,EXPIRED,10409018800,11/07/2018 12:00:00 AM,2018-12-11 00:00:00.000000,12/11/2019 12:00:00 AM,3715 WEST END AVE,NASHVILLE,TN,37205,By making this application for Short Term Rent...,"WEHRLY HOLDINGS, LLC",CASR,24,201807038910409018800,24.0,['Station 13' 'Station 17']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,2018010922,Legree Graham,Legree Graham,Short Term Rental � Multifamily,ISSUED,09215012700,02/27/2018 12:00:00 AM,2018-03-06 00:00:00.000000,03/06/2020 12:00:00 AM,2110 ELLISTON PL 203-21,NASHVILLE,TN,37203,By making this application for a Type 3 Reside...,"I & G ELLISTON, INC.",CASR,21,201801092209215012700,21.0,['Station 11' 'Station 19']
819,2018060708,Kevin Douglas,Kevin Douglas,Short Term Rental - Owner Occupied,EXPIRED,08106040700,09/24/2018 12:00:00 AM,2018-10-09 00:00:00.000000,10/09/2019 12:00:00 AM,1828 25TH AVE N,NASHVILLE,TN,37208,By making this application for a Residential S...,"HELMS, CHARMAINE",CASR,21,201806070808106040700,21.0,['Station 11' 'Station 19']
820,2017072307,None,Kevin Henderson,Short Term Rental � Multifamily,ISSUED,104080L10500CO,11/08/2017 12:00:00 AM,2018-02-02 00:00:00.000000,02/02/2020 12:00:00 AM,1504 18TH AVE S 105,NASHVILLE,TN,37212,By making this application for a Type 3 Reside...,"FERNLAND RENTALS, LLC",CASR,17,2017072307104080L10500CO,17.0,['Station 12' 'Station 8' 'Station 5']
821,2018044963,Curtis Ortmeier,Curtis Ortmeier,Short Term Rental - Owner Occupied,ISSUED,00700020800,07/31/2018 12:00:00 AM,2018-08-07 00:00:00.000000,08/07/2020 12:00:00 AM,2099 TINNIN RD,GOODLETTSVILLE,TN,37072,By making this application for a Residential S...,"ORTMEIER, CURTIS & HOLLY",CASR,10,201804496300700020800,10.0,[]


In [34]:
two_tables_pd = pd.read_sql_query("SELECT DISTINCT(station_dist.district), station_dist.station_number, resident_survey_2018.fire_and_ems__overall, resident_survey_2018.fire_services, resident_survey_2018.fire_response_time, resident_survey_2018.fire_professionalism  FROM strp_2018 INNER JOIN resident_survey_2018 ON resident_survey_2018.zip_code = strp_2018.zip LEFT JOIN station_dist ON station_dist.district = strp_2018.council_district ORDER BY station_dist.district", sql.connect('multi_table_2.db'))


In [35]:
two_tables_pd#.drop_duplicates('district').sort_values('district').reset_index(drop=True)

,district,station_number,fire_and_ems__overall,fire_services,fire_response_time,fire_professionalism
0,NaN,None,Satisfied,Don't Know,Don't Know,Don't Know
1,NaN,None,Satisfied,Satisfied,Satisfied,Satisfied
2,1.0,['Station 30'],Satisfied,Satisfied,Satisfied,Satisfied
3,2.0,['Station 24' 'Station 25'],Satisfied,Satisfied,Don't Know,Don't Know
4,2.0,['Station 24' 'Station 25'],Satisfied,Satisfied,Satisfied,Satisfied
5,3.0,['Station 39'],Very Satisfied,Don't Know,Don't Know,Don't Know
6,3.0,['Station 39'],Satisfied,Satisfied,Don't Know,Don't Know
7,4.0,[],Very Satisfied,Satisfied,Satisfied,Satisfied
8,5.0,['Station 3'],Satisfied,Satisfied,Don't Know,Don't Know
9,5.0,['Station 3'],Satisfied,Satisfied,Satisfied,Don't Know


SELECT O.OrderNumber, CONVERT(date,O.OrderDate) AS Date, 
       P.ProductName, I.Quantity, I.UnitPrice 
  FROM [Order] O 
  JOIN OrderItem I ON O.Id = I.OrderId 
  JOIN Product P ON P.Id = I.ProductId
ORDER BY O.OrderNumber


SELECT station_dist.district, station_dist.station_number, survey.fire_and_ems__overall, survey.fire_services, survey.fire_response_time, survey.fire_professionalism  FROM strp  INNER JOIN survey ON survey.zip_code = strp.zip  LEFT JOIN station_dist ON station_dist.dist = strp.zip